## Data Engineering Capstone Project
#### Introduction
This project will bring together criminal facts of data and distinct property information in the same locality, São Paulo / Brazil, so that the result of this project is a clean and reliable basis and can be used for statist modeling and business intelligence.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
import os
import gc
import logging
from datetime import datetime
from sys import stdout
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

In [43]:
def normalize_columns_name(_df):
    """
    Recive Spark data frame and return data frame with clean column names
    """
    cols = _df.columns
    df_result = _df
    for c in cols:
        df_result = df_result.withColumnRenamed(c, c.lower().replace(' ', '_'))
    
    return df_result

def replace_ncols(_df):
    """
    Recive Spark data frame and return data frame with replaced null string values
    """
    col_types = _df.dtypes
    df_result = _df
    
    for item in col_types:
        if item[1] == 'string':
            df_result = df_result.withColumn(item[0]\
                                            ,f.when(f.col(item[0]).isin('NULL','NaN', 'NA'), None)\
                                            .otherwise(f.col(item[0])) )
    return df_result
    

In [3]:
# Enable garbage collector in Py4j
gc.enable()
JOB_NAME = 'spark_dend_capstone'

In [4]:
spark = SparkSession\
    .builder\
    .appName(JOB_NAME)\
    .master("local[*]")\
    .enableHiveSupport()\
    .getOrCreate()

In [5]:
# Due to the verbosity of log Spark, just leave related error messages org and akka
logger_jvm = spark._jvm.org.apache.log4j
logger_jvm.LogManager.getLogger("org"). setLevel(logger_jvm.Level.ERROR)
logger_jvm.LogManager.getLogger("akka").setLevel(logger_jvm.Level.ERROR)

### Step 1: Scope the Project and Gather Data

#### Describe and Gather Data
1. **Crime Data in Brazil:** comes from [Kaggle](https://www.kaggle.com/inquisitivecrow/crime-data-in-brazil). All crime data from 10 years of police work in Sao Paulo - Brazil. There are more than 16kk of records with several attributes of criminal fact.
3. **Current Properati Listing Information:** comes from [Kaggle](https://www.kaggle.com/properati-data/properties). Property attributes of 1.5 million Latin American listings.

#### Gather Data
The scope of this project is to create a task Spark responsible for entering the data warehouse with clean, reliable data in a dimensional model.
Our analytical repository can be used by data scientists and market intelligence analysts to identify patterns and correlations between datasets.
For example, certain company's real estate branch need to apply a pricing study properties, using data provided by our repository is possible to identify if the surroundings of the properties have a lot indicidencia crimes and what the relevance of these criminal acts.

In [32]:
READ_PATH_CRIME_DATA = './data/crime_data_br/*.csv'
READ_PATH_NEWS = './data/news_folhauol/*.csv'
READ_PATH_PROPERTIES = './data/properties_br/*.csv'

#### Read dataset Crime data in Brazil and drop tuples duplicates

In [44]:
df_crime_crude = spark.read\
    .option('mergeSchema', 'true')\
    .option('header', 'true')\
    .option('inferSchema', 'true')\
    .csv(READ_PATH_CRIME_DATA)\
    .dropDuplicates()

df_crime_crude.printSchema()

root
 |-- NUM_BO: string (nullable = true)
 |-- ANO_BO: string (nullable = true)
 |-- ID_DELEGACIA: string (nullable = true)
 |-- NOME_DEPARTAMENTO: string (nullable = true)
 |-- NOME_SECCIONAL: string (nullable = true)
 |-- DELEGACIA: string (nullable = true)
 |-- NOME_DEPARTAMENTO_CIRC: string (nullable = true)
 |-- NOME_SECCIONAL_CIRC: string (nullable = true)
 |-- NOME_DELEGACIA_CIRC: string (nullable = true)
 |-- ANO: string (nullable = true)
 |-- MES: string (nullable = true)
 |-- DATA_OCORRENCIA_BO: string (nullable = true)
 |-- HORA_OCORRENCIA_BO: string (nullable = true)
 |-- FLAG_STATUS13: string (nullable = true)
 |-- RUBRICA: string (nullable = true)
 |-- DESDOBRAMENTO: string (nullable = true)
 |-- CONDUTA: string (nullable = true)
 |-- LATITUDE: string (nullable = true)
 |-- LONGITUDE: string (nullable = true)
 |-- CIDADE: string (nullable = true)
 |-- LOGRADOURO: string (nullable = true)
 |-- NUMERO_LOGRADOURO: string (nullable = true)
 |-- FLAG_STATUS22: string (nullabl

#### Renamed columns to lower case

In [45]:
df_crime_crude = normalize_columns_name(df_crime_crude)

In [ ]:
qtd_reg_crimes = df_crime_crude.count()
print('Total records df_crime_crude:', qtd_reg_crimes)

### Step 2: Explore and Assess the Data

#### Replace strings 'NULL','NaN', 'NA' to null type

In [46]:
df_crime_crude = replace_ncols(df_crime_crude)

#### The columns are recorded as _col# records of badly formatted strings to our project records affected by these anomalies will be discarded because they represent a very low percentage.

In [47]:
df_crime = df_crime_crude.where('(_c30 is null and _c31 is null and _c32 is null)').drop('_c30', '_c31', '_c32')

In [48]:
df_crime.limit(10).toPandas()

,num_bo,ano_bo,id_delegacia,nome_departamento,nome_seccional,delegacia,nome_departamento_circ,nome_seccional_circ,nome_delegacia_circ,ano,...,logradouro,numero_logradouro,flag_status22,descr_tipo_pessoa,cont_pessoa,sexo_pessoa,idade_pessoa,cor,descr_profissao,descr_grau_instrucao
0,3807,2009,30307,DEMACRO,DEL.SEC.MOGI DAS CRUZES,02º D.P. MOGI DAS CRUZES,DEMACRO,DEL.SEC.MOGI DAS CRUZES,02º D.P. MOGI DAS CRUZES,2009,...,AV ANCHIETA,0,C,Vítima,1,M,41,Preta,APOSENTADO(A),1 Grau incompleto
1,2897,2009,10354,DECAP,DEL.SEC.8º SAO MATEUS,54º D.P. CID. TIRADENTES,DECAP,DEL.SEC.8º SAO MATEUS,54º D.P. CID. TIRADENTES,2009,...,DOUTOR GUILERME DE ABREU SODRE,790,C,Indiciado,6,M,None,None,None,None
2,2919,2009,10344,DECAP,DEL.SEC.6º SANTO AMARO,80º D.P. VILA JOANIZA,DECAP,DEL.SEC.2º SUL,35º D.P. JABAQUARA,2009,...,AC AV ENG ARMANDO DE ARRUDA PEREIRA,4678,C,Vítima,2,F,29,Branca,CONSULTOR(A),Superior completo
3,3744,2009,10365,DECAP,DEL.SEC.4º NORTE,90º D.P. PQ. NOVO MUNDO,DECAP,DEL.SEC.4º NORTE,90º D.P. PQ. NOVO MUNDO,2009,...,AC MARGINAL DIREITA DO TIETE,18,C,Partes,5,M,45,Branca,ANALISTA DE SISTEMAS,Superior completo
4,3363,2009,10308,DECAP,DEL.SEC.5º LESTE,52º D.P. PARQUE S.JORGE,DECAP,DEL.SEC.4º NORTE,19º D.P. VILA MARIA,2009,...,AC PONTE PRES JANIO QUADROS-AV MORVAN DI,0,C,Testemunha,3,M,41,Branca,AJUDANTE,1 Grau incompleto
5,2373,2009,130412,DEINTER 1 - SAO JOSE DOS CAMPOS,DEL.SEC.TAUBATÉ,01º D.P. PINDAMONHANGABA,DECAP,DEL.SEC.3º OESTE,46º D.P. PERUS,2009,...,AL AGENOR COUTO DE MAGALHAES,0,C,Vítima,1,F,63,Branca,None,None
6,2286,2009,30609,DEMACRO,DEL.SEC.CARAPICUIBA,02º D.P. BARUERI,DEMACRO,DEL.SEC.CARAPICUIBA,01º D.P. BARUERI,2009,...,AL ARAGUAIA,1850,C,Vítima,2,M,16,Parda,ESTUDANTE,1 Grau incompleto
7,4665,2009,10342,DECAP,DEL.SEC.1º CENTRO,78º D.P. JARDINS,DECAP,DEL.SEC.1º CENTRO,78º D.P. JARDINS,2009,...,AL FRANCA,184,C,Representante,1,F,36,Branca,EMPRESARIO(A),2 Grau completo
8,3205,2009,10333,DECAP,DEL.SEC.3º OESTE,33º D.P. PIRITUBA,DECAP,DEL.SEC.1º CENTRO,78º D.P. JARDINS,2009,...,AL FRANCA,0,C,Indiciado,1,M,None,Branca,None,None
9,4638,2009,10341,DECAP,DEL.SEC.1º CENTRO,77º D.P. SANTA CECILIA,DECAP,DEL.SEC.1º CENTRO,77º D.P. SANTA CECILIA,2009,...,AL NOTHMANN,100,C,Indiciado,2,M,26,Parda,None,None


#### Normalizing the sexo_pessoa stands for F (female)

In [49]:
df_crime = df_crime.withColumn('sexo_pessoa'\
                               , f.when(f.col('sexo_pessoa') == 'I', 'F')\
                               .otherwise(f.col('sexo_pessoa')))

#### Data quality check: validation if columns have only numeric data, otherwise values will be null

In [80]:
df_crime.select('latitude', 'longitude').where("latitude rlike '[a-z]'").distinct().show(10, False)

+--------+---------+
|latitude|longitude|
+--------+---------+
+--------+---------+



In [50]:
df_crime.select('flag_status13', 'flag_status22').show(30)

+-------------+-------------+
|flag_status13|flag_status22|
+-------------+-------------+
|            C|            C|
|            C|            C|
|            C|            C|
|            C|            C|
|            C|            C|
|            C|            C|
|            C|            C|
|            C|            C|
|            C|            C|
|            C|            C|
|            C|            C|
|            C|            C|
|            C|            C|
|            C|            C|
|            C|            C|
|            C|            C|
|            C|            C|
|            C|            C|
|            C|            C|
|            C|            C|
|            C|            C|
|            C|            C|
|            C|            C|
|            C|            C|
|            C|            C|
|            C|            C|
|            C|            C|
|            C|            C|
|            C|            C|
|            C|            C|
+---------

#### Read dataset Properties

In [4]:
df_properties_crude = spark.read\
    .option('mergeSchema', 'true')\
    .option('header', 'true')\
    .option('quote', '"')\
    .option('escape', '"')\
    .csv(READ_PATH_PROPERTIES)
df_properties_crude = normalize_columns_name(df_properties_crude)

In [5]:
df_properties = df_properties_crude.where("lat-lon is not null")

In [11]:
df_properties.select('price_usd_per_m2','price_per_m2').limit(10).toPandas()

,price_usd_per_m2,price_per_m2
0,2877.2106944444445,9444.444444444445
1,3595.199440993789,11801.242236024846
2,3026.4511111111115,10000.0
3,2044.1819298245614,6754.3859649122805
4,1089.52242,3600.0
5,590.5269512195122,1951.219512195122
6,590.5269512195122,1951.219512195122
7,1101.8389671361504,3615.023474178404
8,4315.673132743363,14159.29203539823
9,2133.5609,7000.0


In [25]:
df_crime.limit(2).toPandas()

,NUM_BO,ANO_BO,ID_DELEGACIA,NOME_DEPARTAMENTO,NOME_SECCIONAL,DELEGACIA,NOME_DEPARTAMENTO_CIRC,NOME_SECCIONAL_CIRC,NOME_DELEGACIA_CIRC,ANO,...,LOGRADOURO,NUMERO_LOGRADOURO,FLAG_STATUS22,DESCR_TIPO_PESSOA,CONT_PESSOA,SEXO_PESSOA,IDADE_PESSOA,COR,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO
0,4,2013,900821,DPPC- DEP.POL.PROT A CIDADANIA,DPPC-DEP.POL.PROT.A CIDADANIA,DIISP - 01ª DEL.POL.,DECAP,DEL.SEC.1º CENTRO,03º D.P. CAMPOS ELISEOS,2009,...,R VINTE E QUATRO DE MAIO,35,C,Declarante,1,F,43,NULL,VENDEDOR(A),2 Grau completo
1,23,2014,30613,DEMACRO,DEL.SEC.CARAPICUIBA,DEL.DEF.MUL. BARUERI,DEMACRO,DEL.SEC.CARAPICUIBA,01º D.P. BARUERI,2009,...,EST DAS ROSAS,0,C,Autor,3,M,49,Parda,EMPRESARIO COMERCIAL,NULL
